In [21]:

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer  # Stemmer específico para português
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import dump


In [16]:
!pip install joblib


In [2]:
df = pd.read_excel('df_reviews_final.xlsx')

In [28]:
# df_reviews_final_sample.csv
df_2 = pd.read_csv('df_reviews_final_sample.csv')
df_2 

,user,review,score,resposta,data,app_name,store,banco
0,Um usuário do Google,Falta pág de salários e FGTS,4,Boa tarde Bárbara.\n\nObrigado pelo feedback. ...,2017-10-06 17:30:03,com.itau.empresas,Google Play,Itaú
1,Um usuário do Google,Bom,5,NaN,2021-06-24 07:34:37,com.itau.empresas,Google Play,Itaú
2,Um usuário do Google,Nao esta abrindo,1,"Bom dia, Giseli! Por favor, você poderia nos e...",2020-11-09 21:50:33,br.com.bradesco.netempresa,Google Play,Bradesco
3,Um usuário do Google,Exelente !!,5,"Boa tarde, Reni! Ficamos felizes em saber que ...",2020-01-31 12:12:10,br.com.bradesco.netempresa,Google Play,Bradesco
4,Um usuário do Google,"prático eficiente e seguro, maravilhoso",4,NaN,2019-05-28 09:32:06,com.itau.empresas,Google Play,Itaú
...,...,...,...,...,...,...,...,...
995,Um usuário do Google,Muito bom,5,NaN,2020-06-20 09:11:38,com.itau.empresas,Google Play,Itaú
996,Um usuário do Google,Facilita muito para acompanharmos a conta,4,NaN,2020-04-26 09:05:10,com.itau.empresas,Google Play,Itaú
997,Um usuário do Google,aplicativo nao atualiza. péssimo,1,"Ei, Claudio! Viemos te ajudar. Para atualizar,...",2021-06-24 06:53:30,com.itau.empresas,Google Play,Itaú
998,Um usuário do Google,Entende como inválidos o itoken tanto do aplic...,1,NaN,2018-02-24 11:07:06,com.itau.empresas,Google Play,Itaú


In [13]:
# Baixar recursos necessários do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')  # Para o stemmer em português

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\henri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\henri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\henri\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [22]:
# Função para limpar e pré-processar o texto
def preprocess_text(text):
    if isinstance(text, str):
        # Converter para minúsculas
        text = text.lower()
        # Remover caracteres especiais e números
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        # Tokenização
        tokens = nltk.word_tokenize(text)
        # Remover stopwords em português
        stop_words = set(stopwords.words('portuguese'))
        tokens = [word for word in tokens if word not in stop_words]
        # Stemming (redução das palavras ao seu radical)
        stemmer = RSLPStemmer()
        tokens = [stemmer.stem(word) for word in tokens]
        # Reconstruir texto
        return ' '.join(tokens)
    else:
        return ''

In [6]:
# Função para categorizar comentários usando regras baseadas em palavras-chave
def categorize_by_keywords(df, text_column):
    """
    Categoriza comentários com base em dicionários de palavras-chave.
    
    Args:
        df: DataFrame contendo os comentários
        text_column: Nome da coluna contendo o texto
    
    Returns:
        DataFrame com a coluna de categoria adicionada
    """
    # Definir categorias e palavras-chave relevantes para app bancário
    keywords_dict = {
        'Problemas de Login': ['login', 'senha', 'acesso', 'entrar', 'chave', 'seguranca', 'token', 'bloquead'],
        'Problemas Técnicos': ['erro', 'fecha', 'bug', 'trava', 'cai', 'instavel', 'lento', 'carrega', 'funcionou'],
        'Usabilidade': ['facil', 'dificil', 'complicad', 'simples', 'intuitivo', 'pratico', 'rapido', 'interface'],
        'Funcionalidades': ['funcao', 'extrato', 'transfer', 'pagamento', 'boleto', 'consulta', 'saldo', 'pix'],
        'Avaliação Geral': ['otimo', 'bom', 'excelente', 'ruim', 'pessimo', 'horrivel', 'melhor', 'pior', 'recomend'],
        'Atualizações': ['atualiza', 'versao', 'nova', 'antigo', 'mudanca', 'melhora']
    }
    
    # Cria uma cópia do DataFrame para não modificar o original
    result_df = df.copy()
    
    # Adiciona uma coluna para a categoria
    result_df['categoria'] = 'Outros'
    
    # Itera sobre cada linha do DataFrame
    for idx, row in result_df.iterrows():
        text = str(row[text_column]).lower()
        
        # Verifica cada categoria
        for category, keywords in keywords_dict.items():
            # Se qualquer palavra-chave estiver presente no texto, atribui a categoria
            if any(keyword in text for keyword in keywords):
                result_df.at[idx, 'categoria'] = category
                break
                
    return result_df

In [7]:
# Função para analisar o sentimento do comentário com base na nota
def add_sentiment(df, score_column):
    """
    Adiciona uma coluna de sentimento com base na nota do comentário.
    
    Args:
        df: DataFrame contendo os comentários
        score_column: Nome da coluna contendo a nota (1-5)
    
    Returns:
        DataFrame com a coluna de sentimento adicionada
    """
    df['sentimento'] = df['score'].apply(lambda x: 
        'Positivo' if x >= 4 else 
        'Neutro' if x == 3 else 
        'Negativo')
    return df

In [8]:

# Função para treinar e avaliar modelos de machine learning
def train_ml_categorization(X_train, X_test, y_train, y_test):
    """
    Treina e avalia diferentes modelos de classificação.
    
    Args:
        X_train, X_test: Features de treino e teste
        y_train, y_test: Labels de treino e teste
    
    Returns:
        O melhor modelo treinado
    """
    # Define modelos para testar
    models = {
        'Naive Bayes': MultinomialNB(),
        'Regressão Logística': LogisticRegression(max_iter=1000),
        'Random Forest': RandomForestClassifier(),
        'SVM': LinearSVC()
    }
    
    best_score = 0
    best_model = None
    
    # Avalia cada modelo
    for name, model in models.items():
        print(f"\nTreinando {name}...")
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        print(f"Acurácia: {score:.4f}")
        
        y_pred = model.predict(X_test)
        print("\nRelatório de Classificação:")
        print(classification_report(y_test, y_pred))
        
        # Salva o melhor modelo
        if score > best_score:
            best_score = score
            best_model = model
    
    return best_model

In [9]:
# Função para visualizar a distribuição das categorias
def plot_categories_distribution(df, category_column):
    plt.figure(figsize=(12, 6))
    sns.countplot(y=df[category_column], order=df[category_column].value_counts().index)
    plt.title('Distribuição das Categorias')
    plt.xlabel('Número de Comentários')
    plt.ylabel('Categoria')
    plt.tight_layout()
    plt.savefig('categorias_distribuicao.png')
    plt.close()

In [25]:
# Função para visualizar cruzamento entre categoria e sentimento
def plot_category_sentiment(df, category_column, sentiment_column):
    plt.figure(figsize=(14, 8))
    ct = pd.crosstab(df[category_column], df[sentiment_column])
    ct_percent = ct.div(ct.sum(axis=1), axis=0)
    ct_percent.plot(kind='barh', stacked=True, colormap='viridis')
    plt.title('Distribuição de Sentimento por Categoria')
    plt.xlabel('Proporção')
    plt.ylabel('Categoria')
    plt.legend(title='Sentimento')
    plt.tight_layout()
    plt.savefig('categoria_sentimento.png')
    plt.close()

In [23]:

# Função principal para processar o dataset
def main():
    # Carrega os dados
    # Ajuste o separador conforme necessário (parece ser um CSV ou TSV)
    try:
        df = pd.read_excel('df_reviews_final.xlsx')
    except:
        # Tente outro separador se o primeiro não funcionar
        df = pd.read_excel('df_reviews_final.xlsx')
    
    # Verifica as colunas disponíveis
    print("Colunas disponíveis no dataset:")
    print(df.columns.tolist())
    
    # Renomeia as colunas se necessário para facilitar o acesso
    # Ajuste isso de acordo com as colunas reais do seu dataset
    column_mapping = {
        'user': 'usuario',
        'review': 'comentario',
        'score': 'nota',
        'resposta': 'resposta_empresa',
        'data': 'data',
        'app_name': 'nome_app',
        'store': 'loja'
    }
    df = df.rename(columns=column_mapping)
    
    # Mostra as primeiras linhas do dataset
    print("\nPrimeiras linhas do dataset:")
    print(df.head())
    
    # Informações básicas
    print("\nInformações básicas sobre o dataset:")
    print(f"Número de comentários: {len(df)}")
    print(f"Distribuição das notas:")
    print(df['nota'].value_counts().sort_index())
    
    # Pré-processamento dos comentários
    print("\nRealizando pré-processamento dos comentários...")
    df['comentario_processado'] = df['comentario'].apply(preprocess_text)
    
    # Adiciona sentimento com base na nota
    df = add_sentiment(df, 'nota')
    
    # ABORDAGEM 1: Categorização baseada em palavras-chave
    print("\n==== ABORDAGEM 1: Categorização por palavras-chave ====")
    categorized_df = categorize_by_keywords(df, 'comentario_processado')
    
    print("Distribuição de categorias:")
    category_counts = categorized_df['categoria'].value_counts()
    print(category_counts)
    
    # Visualiza a distribuição das categorias
    plot_categories_distribution(categorized_df, 'categoria')
    
    # Visualiza relação entre categoria e sentimento
    plot_category_sentiment(categorized_df, 'categoria', 'sentimento')
    
    # ABORDAGEM 2: Categorização por machine learning
    print("\n==== ABORDAGEM 2: Categorização por machine learning ====")
    
    # Usar as categorias da abordagem por palavras-chave como rótulos para treinar o modelo
    X = categorized_df['comentario_processado']
    y = categorized_df['categoria']
    
    # Divide em conjuntos de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Vetorização do texto usando TF-IDF
    print("\nVetorizando os textos...")
    vectorizer = TfidfVectorizer(max_features=5000)
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    # Treina e avalia os modelos
    best_model = train_ml_categorization(X_train_vec, X_test_vec, y_train, y_test)
    
    # Cria um pipeline com o melhor modelo
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000)),
        ('classifier', best_model)
    ])
    
    # Treina o pipeline com todos os dados
    pipeline.fit(X, y)
    
    # Exemplo de aplicação: categorizar novos comentários
    print("\n==== APLICAÇÃO: Categorizar novos comentários ====")
    
    # Exemplos de novos comentários (ajuste conforme necessário)
    novos_comentarios = [
        "Não consigo fazer login, pede a chave de segurança mas não reconhece",
        "App muito bom, fácil de usar e com todas as funções que preciso",
        "Fiz a atualização e agora fica travando quando tento fazer transferência",
        "Não consigo ver meu extrato, dá erro toda vez que tento acessar",
        "Excelente aplicativo, muito prático para gerenciar minha empresa"
    ]
    
    # Pré-processa os novos comentários
    novos_comentarios_proc = [preprocess_text(texto) for texto in novos_comentarios]
    
    # Prevê as categorias
    categorias_previstas = pipeline.predict(novos_comentarios_proc)
    
    # Exibe os resultados
    print("\nResultados da categorização:")
    for comentario, categoria in zip(novos_comentarios, categorias_previstas):
        print(f"Comentário: {comentario}")
        print(f"Categoria prevista: {categoria}\n")
    
    # Salva o modelo treinado
    dump(pipeline, 'modelo_categorizacao_app_bradesco.joblib')
    print("Modelo salvo como 'modelo_categorizacao_app_bradesco.joblib'")
    
    # Salva o DataFrame categorizado
    categorized_df.to_csv('comentarios_categorizados.csv', index=False, encoding='utf-8-sig')
    
    print("Dataset categorizado salvo como 'comentarios_categorizados.csv'")

    # Dicas de insights que podem ser extraídos
    print("\n==== INSIGHTS POTENCIAIS ====")
    print("1. Categorias mais comuns de problemas reportados")
    print("2. Relação entre categorias e sentimento (notas)")
    print("3. Evolução temporal das categorias (análise de tendência)")
    print("4. Comparação entre versões do aplicativo")
    print("5. Palavras mais frequentes em cada categoria")

if __name__ == "__main__":
    main()

Colunas disponíveis no dataset:
['user', 'review', 'score', 'resposta', 'data', 'app_name', 'store']

Primeiras linhas do dataset:
                usuario                                         comentario  \
0  Um usuário do Google  Da erro e fecha sozinho ,não puxa extrato só d...   
1  Um usuário do Google                                     Muito ótimoooo   
2  Um usuário do Google                                        Excelente 👏   
3  Um usuário do Google                        Facil de trabalhar com ele.   
4  Um usuário do Google  Nao funcionou. Pede a chave de seguranca que j...   

   nota resposta_empresa                data                    nome_app  \
0     1              NaN 2025-03-11 20:23:03  br.com.bradesco.netempresa   
1     5              NaN 2025-03-11 19:07:50  br.com.bradesco.netempresa   
2     5              NaN 2025-03-11 17:06:38  br.com.bradesco.netempresa   
3     5              NaN 2025-03-11 16:26:26  br.com.bradesco.netempresa   
4     1             

c:\Users\henri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\henri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\henri\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                    precision    recall  f1-score   support

      Atualizações       0.00      0.00      0.00        51
   Avaliação Geral       0.98      0.89      0.93      9197
   Funcionalidades       0.85      0.54      0.66      1426
            Outros       0.87      1.00      0.93     21986
Problemas Técnicos       0.98      0.15      0.27       639
Problemas de Login       0.98      0.34      0.50       858
       Usabilidade       0.99      0.52      0.68      1243

          accuracy                           0.90     35400
         macro avg       0.81      0.49      0.57     35400
      weighted avg       0.91      0.90      0.89     35400


Treinando Regressão Logística...
Acurácia: 0.9812

Relatório de Classificação:
                    precision    recall  f1-score   support

      Atualizações       1.00      0.14      0.24        51
   Avaliação Geral       0.99      0.98      0.98      9197
   Funcionalidades       0.96      0.93      0.94      1426
            Outr

c:\Users\henri\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Acurácia: 0.9913

Relatório de Classificação:
                    precision    recall  f1-score   support

      Atualizações       1.00      0.41      0.58        51
   Avaliação Geral       1.00      0.99      0.99      9197
   Funcionalidades       0.98      0.97      0.98      1426
            Outros       0.99      1.00      0.99     21986
Problemas Técnicos       0.99      0.91      0.95       639
Problemas de Login       1.00      0.96      0.98       858
       Usabilidade       0.99      0.96      0.98      1243

          accuracy                           0.99     35400
         macro avg       0.99      0.89      0.92     35400
      weighted avg       0.99      0.99      0.99     35400



c:\Users\henri\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(



==== APLICAÇÃO: Categorizar novos comentários ====

Resultados da categorização:
Comentário: Não consigo fazer login, pede a chave de segurança mas não reconhece
Categoria prevista: Problemas de Login

Comentário: App muito bom, fácil de usar e com todas as funções que preciso
Categoria prevista: Avaliação Geral

Comentário: Fiz a atualização e agora fica travando quando tento fazer transferência
Categoria prevista: Funcionalidades

Comentário: Não consigo ver meu extrato, dá erro toda vez que tento acessar
Categoria prevista: Outros

Comentário: Excelente aplicativo, muito prático para gerenciar minha empresa
Categoria prevista: Outros

Modelo salvo como 'modelo_categorizacao_app_bradesco.joblib'
Dataset categorizado salvo como 'comentarios_categorizados.csv'

==== INSIGHTS POTENCIAIS ====
1. Categorias mais comuns de problemas reportados
2. Relação entre categorias e sentimento (notas)
3. Evolução temporal das categorias (análise de tendência)
4. Comparação entre versões do aplicati

<Figure size 1400x800 with 0 Axes>

In [ ]:
quero conseguir categorizar comentários que são campos textuais abertos, quero categorizar em categorias com python


In [26]:
df

,user,review,score,resposta,data,app_name,store
0,Um usuário do Google,"Da erro e fecha sozinho ,não puxa extrato só d...",1,NaN,2025-03-11 20:23:03,br.com.bradesco.netempresa,Google Play
1,Um usuário do Google,Muito ótimoooo,5,NaN,2025-03-11 19:07:50,br.com.bradesco.netempresa,Google Play
2,Um usuário do Google,Excelente 👏,5,NaN,2025-03-11 17:06:38,br.com.bradesco.netempresa,Google Play
3,Um usuário do Google,Facil de trabalhar com ele.,5,NaN,2025-03-11 16:26:26,br.com.bradesco.netempresa,Google Play
4,Um usuário do Google,Nao funcionou. Pede a chave de seguranca que j...,1,NaN,2025-03-11 12:40:39,br.com.bradesco.netempresa,Google Play
...,...,...,...,...,...,...,...
117994,RaRaull,Cadastrei a senha eletrônica com 6 dígitos. Ag...,1,"Oi, Raul. Você conseguiu atualizar a senha e u...",2018-07-04 17:05:03,Safra Empresas,Apple Store
117995,Alan Moreira,"App “ok”, muitos bugs e plataforma pouco intui...",3,NaN,2025-03-10 21:58:29,Safra Empresas,Apple Store
117996,Rtunoda,"Não funciona com 3g, só wifi. E não permite au...",1,Olá. Disponibilizamos uma atualização do aplic...,2012-05-26 16:45:17,Safra Empresas,Apple Store
117997,Edu.05,Precisa melhorar tudo,1,"Oi, Edu. Disponibilizamos uma atualização do a...",2018-10-26 15:03:49,Safra Empresas,Apple Store


In [27]:
categorized_df

NameError: name 'categorized_df' is not defined